#### "IMAGE" Mask Example

In this example, we use an image mask (created in an image editing app) visualize a necklace set on the woman in the source image.

**Input**:

<div style="display: flex; gap: 2em; margin-right: 2em">
  <div style="text-align: center; flex: 1">
    <img src="../images/woman-closeup.png" style="border: 1px solid #99999930">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="text-align: center; flex: 1">
    <img src="../images/woman-closeup-necklace_mask.png" style="border: 1px solid #99999930">
    <strong><tt>maskImage</tt></strong>
  </div>
  <div style="text-align: center; flex: 1">
    <img src="../images/necklace-1.png">
    <strong><tt>referenceImage</tt></strong>
  </div>
</div>

**Output**:

<div style="text-align: center; flex: 1; max-width: 500px">
    <img src="images/example-output/mask_type-image-output.png" style="border: 1px solid #99999930">
    <strong>Output</strong>
  </div>


In [ ]:
import logging
import os
from datetime import datetime
from random import randint
from amazon_nova_canvas_utils import (
    generate_images,
    load_image_as_base64,
    base64_to_pil_image,
)

logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_image_mask_source.jpg"
mask_image_path = "../images/vto-images/vto_image_mask_maskimage.png"
reference_image_path = "../images/vto-images/vto_image_mask_reference.jpg"

inference_params = {
    "taskType": "VIRTUAL_TRY_ON",
    "virtualTryOnParams": {
        "sourceImage": load_image_as_base64(source_image_path),
        "referenceImage": load_image_as_base64(reference_image_path),
        # Use the "IMAGE" mask type
        "maskType": "IMAGE",
        # Configure the image mask
        "imageBasedMask": {"maskImage": load_image_as_base64(mask_image_path)},
        # When the try-on item is small, using the "DETAILED" merge style is
        # almost always the best choice.
        "mergeStyle": "DETAILED",
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 6.5,
        "seed": randint(0, 2147483646),
    },
}

output_folder = os.path.join("output", datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))

try:
    response_body = generate_images(
        inference_params=inference_params,
        save_folder_path=output_folder,
        model_id="amazon.nova-canvas-v1:0",
        region_name="us-east-1",
    )

    # An error message may be returned, even if some images were generated.
    if "error" in response_body:
        logging.error(response_body["error"])

    if "images" in response_body:
        # Display all images.
        for image_base64 in response_body["images"]:
            image = base64_to_pil_image(image_base64)
            display(image)

except Exception as e:
    logging.error(e)

print(f"Done! Artifacts saved to {os.path.abspath(output_folder)}")